# My Hack: Classifying mergers in Eagle simulations

What are eagle simulations?
why eagle? 
long term goal 
short term goal 

## Classifying mergers by their Size and Mass Ratio

### Training set 

<img src="Images/EM.png" width=800 height=800 />

### Procedure

<img src="Images/classification.png" width=800 height=800 />

Links to the data - https://drive.google.com/drive/folders/1AMv2tWMj11mR5O2KA86Ab4z9XQZRBD5r?usp=sharing

## Code

In [ ]:
def data(path, N, parameter): 
    ''''
    Retrieving the galaxy merger images and their corresponding labels of size and mass ratio
    
    i/p params:
    
    path - path of the images and labels
    N - number of images we want in the training set
    parameter - Mass/Size ratio 
    
    o/p params:
    
    X - an np 3D array of randomly picked 'N' galaxy merger images. The 3D is to comply with the dimensional 
    requirement of the neural network. Here, the images are black and white so all 3 dimensions are the same. 
    Y - labels 
    
    ''''
    
    X =[]
    Y = np.array([])
    redshift=[]
    merger=[]

    for path, subdirs, image_names in os.walk(path+ '/eagle_images'):
        image_names=np.array(image_names)
        index_store= np.where(image_names=='.DS_Store')[0]
        image_names=np.delete(image_names,index_store)
        ind=np.random.choice(np.arange(len(image_names)),N)   #indices of N randomly picked images 
        picked_image_names= image_names[ind]                  #images pertaining to these indices 
        
        for i, name in enumerate(picked_image_names):
            if name.endswith('.jpg'): 
                img_path = os.path.join(path,name)
                img_pixels = np.array(Image.open(img_path).getdata())
                X.append(img_pixels)
                redshift.append(int(name.split('_')[1]))
                merger.append(int(name.split('_')[2]))
                f= h5py.File(path+'/mergers_identified/mergers_'+str(int(redshift[-1]))+'.hdf5', 'r')
                Y=list(np.append(Y,f.get(parameter).value[int(merger[-1])]))
    
    X=np.array(X)
    X=X.reshape((len(Y),224,224,3))
    Y=np.array(Y)
    
    return X,Y

In [ ]:
X, Y = data(path, 100, parameter= 'Size Ratio')

In [ ]:
print(np.shape(X))
print(np.shape(y))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)#, random_state=seed)

In [ ]:
print(len(XX_train))
print(len(XX_test))
print(len(Y_train))
print(len(Y_test))

In [ ]:
XX_train=np.array(XX_train)
Y_train=np.array(Y_train)
XX_test=np.array(XX_test)
Y_test=np.array(Y_test)

In [ ]:
import tensorflow
from tensorflow.python.keras import layers 
from tensorflow.python.keras.losses import categorical_crossentropy
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.python.keras import backend
from tensorflow.keras.models import load_model
from keras.utils import CustomObjectScope
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.metrics import top_k_categorical_accuracy

In [ ]:
Yround_train= np.array([])
Yround_test= np.array([]) 

cl=[0,0.25,0.5,0.75,1.0,1.25]

Y_train_ohe = np.zeros((len(Y_train), 5), dtype=np.int)
Y_test_ohe= np.zeros((len(Y_test), 5), dtype=np.int)

for i in np.arange(len(Y_train)):
    for j in np.arange(1,len(cl)):
        if Y_train[i]> cl[j-1] and  Y_train[i]< cl[j]: 
            Yround_train=np.append(Yround_train,cl[j])
            Y_train_ohe[i, j-1] = 1

for i in np.arange(len(Y_test)):
    for j in np.arange(1,len(cl)):
        if Y_test[i]> cl[j-1] and  Y_test[i]< cl[j]: 
            Yround_test=np.append(Yround_test,cl[j])
            Y_test_ohe[i, j-1] = 1

In [ ]:
Yround_train,Yround_test, Y_train_ohe, Y_test_ohe

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense
from tensorflow.keras import layers,models
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.utils import to_categorical

In [ ]:
def mnist(): 
    
    def mnistmodel(inputs): 

        """
        Creates and returns neural net model
        """

        x = layers.Conv2D(32, kernel_size=(3, 3), 
                          activation='relu',
                          padding='valid',
                          data_format=backend.image_data_format(),
                          input_shape=(224,224,3))(inputs)
        x = layers.Dropout(0.5)(x)
        x = layers.Conv2D(32, kernel_size=(3, 3), activation='relu',padding='valid')(x)
        x = layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2))(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Flatten()(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        #x = layers.Dense(self.nclasses, activation='sigmoid')(x)
        x = layers.Dense(5, activation='softmax')(x)
        return x

    model_input = layers.Input(shape=(224,224,3))
    model_output = mnistmodel(model_input)
    model = Model(inputs=model_input, outputs=model_output)
    return model

model= mnist()
BS=2

sgd=optimizers.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss=categorical_crossentropy,optimizer=sgd,metrics=['accuracy'])

callbacks=[TensorBoard(log_dir='/home/vasist/Tensorboard/TB_resnet50_'+str(100), batch_size=BS), ModelCheckpoint('/home/vasist/Model/M_resnet50_'+str(100),monitor='val_acc',verbose=1,period=1)]  #-{val_accuracy:.2f}

# construct the training image generator for data augmentation
aug = ImageDataGenerator(horizontal_flip=True)#, rotation_range=20, zoom_range=0.15,
#width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, fill_mode="nearest")

# train the network
# validation_steps= int((len(XX_test)+BS-1)/BS)-1
# train_steps=int((len(XX_train)+BS-1)/BS)-1

validation_steps= len(XX_test)//BS
train_steps=len(XX_train)//BS
print(validation_steps,train_steps)

H = model.fit_generator(aug.flow(XX_train, Y_train_ohe, batch_size=BS),
validation_data=aug.flow(XX_test, Y_test_ohe), steps_per_epoch=train_steps,
epochs=100,callbacks=callbacks,validation_steps=validation_steps)


In [ ]:
# for x,y in aug.flow(XX_train, Y_train_ohe,batch_size=3): 
#     fig=plt.figure(figsize=(15,7))
#     for i in range(0, len(x)):
#         plt.subplot(330 + 1 + i)
# #         if x[i] : 
#         Tx=x[i].T[0]
#         print(cl[np.nonzero(y[i])[0][0]+1])
#         plt.imshow(Tx)
#     plt.show()
#     if len(x)<3: 
#         break